In [1]:
import cv2
import numpy as np

In [2]:
def find_text_contours(image):
    """
    이미지에서 글씨 윤곽선을 찾아 반환합니다.
    
    Args:
        image: 입력 이미지 (BGR 채널)
    
    Returns:
        글씨 윤곽선 리스트
    """
    
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (5, 5), 0)
    gray = cv2.bitwise_not(gray) # 객체보다 배경이 밝은 경우 이미지 반전
    
    
    # Canny 에지 검출 알고리즘 적용
    edge = cv2.Canny(gray, 100, 100)
    _, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)
    adaptive_threshold= cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)

    thresh = cv2.erode(thresh, None, iterations=2)
    thresh = cv2.dilate(thresh, None, iterations=2)
    
    # 윤곽선 추출
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    return contours

In [6]:
def mark_text_regions(image, contours, min_distance):
    """
    이미지에 글씨 영역을 사각형으로 표시합니다.
    
    Args:
        image: 입력 이미지 (BGR 채널)
        contours: 글씨 윤곽선 리스트
    
    Returns:
        글씨 영역이 표시된 이미지
    """
	
	# 박스를 표기하고 거리를 계산합니다.
	boxes = []
	for contour in contours:
		x, y, w, h = cv2.boundingRect(contour)
		box = (x, y, x+w, y+h)
		boxes.append(box)
	
	print(boxes)
	
	
	# 거리가 좁은 박스를 하나의 박스로 합칩니다.
	merged_boxes = []
	for i in range(len(boxes)):
		merged_box = boxes[i]
		
		for j in range(i+1, len(boxes)):
			if is_close_boxes(merged_box, boxes[j], min_distance) == True:
				merged_box = merge_boxes(merged_box, boxes[j])
				merged_boxes.append(merged_box)
			
	print(merged_boxes)
	
	# 합쳐진 박스를 표기합니다.
	for box in merged_boxes:
	cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)
	
	return image

TabError: inconsistent use of tabs and spaces in indentation (1705671830.py, line 14)

In [ ]:
def is_close_boxes(box1, box2, min_distance):
    """
    두 박스의 거리가 좁은지 확인합니다.
    
    Args:
        box1: 첫 번째 박스
        box2: 두 번째 박스
        min_distance: 최소 거리
    
    Returns:
        True if 두 박스의 거리가 좁으면, False otherwise
    """

    b1_x, b1_y, b1_w, b1_h = box1
    
    box1_center_x = (b1_x + b1_w) / 2
    box1_center_y = (b1_y + b1_h) / 2

    
    b2_x, b2_y, b2_w, b2_h = box2

    box2_center_x = (b2_x + b2_w) / 2
    box2_center_y = (b2_y + b2_h) / 2
    
    # 두 박스의 중심점 간의 거리를 계산합니다.
    distance = np.sqrt(
        ((box1_center_x - box1_center_y)**2) +
        ((box2_center_x - box2_center_y)**2)
    )

    return distance <= min_distance

In [ ]:
def merge_boxes(box1, box2):
    """
    두 박스를 하나의 박스로 합칩니다.
    
    Args:
        box1: 첫 번째 박스
        box2: 두 번째 박스
    
    Returns:
        합쳐진 박스
    """
    
    b1_x, b1_y, b1_w, b1_h = box1
    b2_x, b2_y, b2_w, b2_h = box2
    
    return (
        min(b1_x, b2_x), # X
        min(b1_y, b2_y), # Y
        max(b1_w, b2_w), # W
        max(b1_h, b2_h), # H
    )

In [ ]:
# 이미지 불러오기
image = cv2.imread("../dataset_dir/13.한국어글자체/01.손글씨/01_handwriting_sentence_images/1_sentence/00000002.png")

# 최소 거리 설정
# min_distance = 10

# 글씨 윤곽선 찾기
contours = find_text_contours(image)

# 글씨 영역 표시
marked_image = mark_text_regions(image, contours, min_distance)

In [ ]:
from time import time

# 결과 이미지 출력
cv2.imwrite(f'./image/untitle10-{time()}.jpg', marked_image)